In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("talk")
plt.rcParams["figure.figsize"] = [9.708,6]
import warnings
warnings.filterwarnings('ignore')
#this is our new one
import tensorflow as tf
from tensorflow.keras.datasets import mnist
tf.random.set_seed(0)
np.random.seed(0)

Check the tensorflow version.  Should be >2.4.

In [ ]:
print(tf.__version__) 

# MNIST

The MNIST dataset is a dataset of 60,000 training and 10,000 test examples of handwritten digits, originally constructed by Yann Lecun, Corinna Cortes, and Christopher J.C. Burges. It is very widely used to check simple methods. There are 10 classes in total ("0" to "9"). This dataset has been extensively studied, and there is a history of methods and feature construc- tions at [https://en.wikipedia.org/wiki/MNIST_database](https://en.wikipedia.org/wiki/MNIST_database) and at the original site, [http://yann.lecun.com/exdb/mnist/](http://yann.lecun.com/exdb/mnist/). You should notice that the best methods perform extremely well.

You will be implementing a simple neutral network and should, with appropriate hyperparameter settings, get a test
error of approximately 5% or better. Specifically, you will implement a neural network with a total of three
layers: an input layer, a hidden layer, and an output layer. Please pay careful attention to the following
implementation details

In [ ]:
num_classes= 10    # 10 digits 
batch_size = 128   # during training to find the gradient
epochs     = 10    # how long to train


## load the data

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
input_shape= (X_train.shape[1],X_train.shape[2])

In [ ]:
print(X_train.shape)

## prepare the data
Flatten the images, convert the class labels, and scale the data.

In [ ]:
class_names = np.unique(y_test)

Each label will have to be transformed to a one-hot-encoding representation, i.e., a vector of length 10
that has a single 1 in the position of the true class and 0 everywhere else

In [ ]:
Y_train = tf.keras.utils.to_categorical(y_train, num_classes)
Y_test  = tf.keras.utils.to_categorical(y_test,  num_classes)

In [ ]:
#normalize the data between 0-1
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32')   / 255
#Reshape To Match The Keras's Expectations
X_train = X_train.reshape(X_train.shape[0], 1, input_shape[0], input_shape[1])
X_test = X_test.reshape(X_test.shape[0], 1,    input_shape[0], input_shape[1])


In [ ]:
#==============
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

## explore data

In [ ]:
sns.countplot(x=y_train)
plt.show();

## View the data
The MNIST dataset contains hand-written digits that are 28x28 pixels large. Here is one image

In [ ]:
jj  = 325 #lets look at image 325
# preview the images first
fig = plt.figure(figsize=(6,6))
imgi = X_train[jj].reshape((input_shape[0],input_shape[1]))
plt.imshow(imgi,interpolation='nearest',cmap=plt.cm.gray)
plt.axis('off')   
plt.show()

Here are a few examples of each of the digits (0-9) in the dataset:

In [ ]:
# preview the images first
fig=plt.figure()
ncols,nrows = len(class_names), 4
fig, axs = plt.subplots(nrows=nrows, ncols=ncols,figsize=(12,5))
print(axs.shape)
for i in range(ncols):
    inn = y_train==i      #mask to select only that class
    Xi  = X_train[inn]    #select only X_train belonging to that class
    for j in range(nrows):        
        imgi = Xi[j].reshape((input_shape[0],input_shape[1]))
        axs[j,i].imshow(imgi,interpolation='nearest',cmap=plt.cm.gray)
        axs[j,i].axis('off')
plt.tight_layout()
fig.subplots_adjust(hspace=.1)
plt.show()

## Digit Recognition

Here is an example of using the network to classify whether the last image contains a small digit. Again, since we haven't trained the network yet, the predicted probability of the image containing a small digit is close to half. The "network" is unsure.


## Simple network
Build and compile a basic model.

Layers for our Network.

* **Input layer** - size 784 
    * flatten the input image (28x28).
* **Hidden layer** - size 100
    * Dense (fully connected) network from input layer to these 128 neuron hidden layer.
* **Output layer** - size 10
    * Dense layer (fully connected back to the 128 neuron hidden layer). The 10 is the number of classes.  Given an input image, our network should **light** up the corresponding neuron of our target.
* **Softmax activation** - convert our output into a probability for each class.

By default, TensorFlow initializes the weights with random (small) values. This allows us to break symmetry
that occurs when all weights are initialized to 0.    (we explore that last today)

In [ ]:
model = tf.keras.models.Sequential([              # model type
  tf.keras.layers.Flatten(input_shape=X_train[1].shape),  # input layer
  tf.keras.layers.Dense(100, activation='relu'),  # hidden layer
  tf.keras.layers.Dense(10),                      # output to each class, could just stop here
  tf.keras.layers.Softmax()                       # convert to probability
])

Use our network to make a prediction.  Since our model is untrained, our network predicts this sample as ~10% to each class. Our network is randomly guessing at the class.

In [ ]:
#our output is the probability to each class
predictions = model(X_train[:1]).numpy()
print(predictions) ###each should be about 10%

## Define our loss function

The **categorical_crossentropy** loss takes a vector of logits and a True index and returns a scalar loss for each example.

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

$$ L = \sum_i ( y_i \ln z_i )+ (1-y_i) \ln (1-z_i) $$
where z is the output of node $i$.



#### NOTE
* If your targets are **one-hot encoded**, use **categorical_crossentropy**. Examples of one-hot encodings: [1,0,0] or [0,1,0]

* If your targets are integers, use **sparse_categorical_crossentropy**. Examples of integer encodings (for the sake of completion).

##### Another Note
Use sparse categorical crossentropy when your classes are mutually exclusive (e.g. when each sample belongs exactly to one class) and categorical crossentropy when one sample can have multiple classes or labels are soft probabilities (like [0.5, 0.3, 0.2]).

#### Loss initial value = 2.3
This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

View our initial loss

In [ ]:
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
loss_fn(Y_train[:1], predictions).numpy()

### define our loss function 

We will be using the Categorical cross-entropy, described above.

### define our optimizer
We will use stochast gradient descent with the following parameters
* Learning rate $\alpha = 0.1$
* Momentum $\beta = 0.9$


In [ ]:
#define our optimizer
sgd = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=False, name='SGD')
#
model.compile(optimizer=sgd,
              loss='categorical_crossentropy', #need to define our loss function
              metrics=['accuracy'])

In order for the network to be useful, we need to actually train it, so that the weights are actually meaningful, non-random values. As we mentioned before, we'll use the network to make predictions, then compare the predictions agains the ground truth. But how do we compare the predictions against the ground truth? We'll need a few more things. In particular, we need to:

1. Specifying a "reward" or a **loss** (negative reward) that judges how good or bad the prediction was, compared to the **ground truth** actual value.
2. Specifying an **optimizer** that tunes the parameters to improve the reward or loss.

Choosing a good **loss function**  L(actual,predicted)  for a problem is not a trivial task. The definition of a loss function also transforms a **classification** problem into an **optimization** problem: what set of parameters minimizes the loss (or maximizes the reward) across the training examples?

Turning a learning problem into an optimization problem is actually a very subtle but important step in many machine learning tools, because it allows us to use tools from mathematical optimization.

That there are **optimizers** that can tune the network parameters for us is also really, really cool. Unfortunately, we won't talk much about optimizers and how they work in this course. You should know though that these optimizers are what makes machine learning work at all.

For now, we will choose a standard loss function for a **binary classification** problem: the **binary cross-entropy loss**. 


## Train the model
We will train this network to perform a digit recognition task. That is, we will use the MNIST dataset of hand-written digits.

1. Predict. Show the network pictures of digits, one by one.  Then we see what the network predicts.
3. Loss. Check the loss function for that example digit, comparing the network prediction against the ground truth
4. Optimize through grandient descent. We make a small update to the parameters to try and improve the loss for that digit.
5. Repeat. Continue doing this many many times.


The Model.fit method adjusts the model parameters to minimize the loss:

In [ ]:
tstart = tf.timestamp()
history = model.fit(X_train, Y_train, 
                    epochs=epochs,batch_size=batch_size,
                    validation_split = 0.2) # Data for evaluation
total_time = tf.timestamp() - tstart
print("total time %3.3f seconds"%total_time)

The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".

In [ ]:
results = model.evaluate(X_test, Y_test, batch_size=128)
print("test loss, test acc:", results)
# Generate predictions (probabilities -- the output of the last layer)
print("Generate predictions for 3 samples")
predictions = model.predict(X_test[:3])
print("predictions shape:", predictions.shape)


The image classifier is now trained to >97% accuracy on this dataset. 


## Graph

In [ ]:
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=True,  show_dtype=False,
    rankdir='TB', expand_nested=False, dpi=150, show_layer_names=True
)

### Summary

In [ ]:
model.summary()

Our simple NN network has the above architecture.  You can see even this relatively small network has a large number of parameters: **79,510**.  So overfitting is always an concern.

## Evaluate

In [ ]:
results_test = model.evaluate(X_test, Y_test, batch_size=128,verbose=0)

In [ ]:
#we will use this a lot, so lets make a function
def printAccuracy(history,results_test):
    print("train loss %.5f \t train acc: %.5f"%(history.history['loss'][-1],history.history['accuracy'][-1]))
    print("valid loss %.5f \t valid acc: %.5f"%(history.history['val_loss'][-1],history.history['val_accuracy'][-1]))
    print("test loss  %.5f \t test acc:  %.5f"%(results_test[0],results_test[1]))

In [ ]:
printAccuracy(history,results_test)

## Plot training loss per epoch

In [ ]:
#we will do this a lot, so lets make a function for this
def plot_result(history,results_test):
    # Get training and validation histories
    training_acc = history.history['accuracy']
    val_acc      = history.history['val_accuracy']
    # Create count of the number of epochs
    epoch_count = range(1, len(training_acc) + 1)
    # Visualize loss history
    plt.plot(epoch_count, training_acc, 'b-o',label='Training')
    plt.plot(epoch_count, val_acc, 'r--',label='Validation')
    plt.plot(epoch_count, results_test[1]*np.ones(len(epoch_count)),'k--',label='Test')
    plt.legend()
    plt.title("Training and validation accuracy")
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')

In [ ]:
plot_result(history,results_test)
plt.title("MNIST 1-hidden layer in %3.2f s"%(total_time)) #overwrite the title 
plt.show()

Above we see the plot for train, validation, and test data. Lets make a few quick observations to better know what is happening when we train the data set.
* The final training accuracy is very high, the network is learning the data - and it looks like its overtraining.  If we keep going and our network is large enough, we expect this loss to go to one.  Its memorizing our training data.
* The final validation and test accuracies are similar.  If its not, then its likely we aren't training long enough.


## Predict

Predict classes on the test set.

In [ ]:
y_hat = np.argmax(model.predict(X_test), axis=-1)
pd.crosstab(y_hat, y_test) #note lower case is the numeric value, upper case (Y_test) is the one-hot encoding

### Misidentified examples

In [ ]:
y_hat = np.argmax(model.predict(X_test), axis=-1)
test_wrong = [im for im in zip(X_test,y_hat,y_test) if im[1] != im[2]]
#=====
#plot
plt.figure(figsize=(10, 10))
for ind, val in enumerate(test_wrong[:100]):
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.subplot(10, 10, ind + 1)
    im = 1 - val[0].reshape((28,28))
    plt.axis("off")
    plt.text(0, 0, val[2], fontsize=14, color='blue')
    plt.text(8, 0, val[1], fontsize=14, color='red')
    plt.imshow(im, cmap='gray')
plt.show()

## total number of connections

In [ ]:
for layer in model.layers:
    print(layer.name, "in:",layer.input_shape,"out:",layer.output_shape)

### Save the model weights

In [ ]:
Wsave = model.get_weights()
model.set_weights(Wsave)

# Dropout

During training, some number of layer outputs are randomly ignored or “dropped out.” This has the effect of making the layer look-like and be treated-like a layer with a different number of nodes and connectivity to the prior layer. In effect, each update to a layer during training is performed with a different “view” of the configured layer.

Dropout has the effect of making the training process noisy, forcing nodes within a layer to take on more or less responsibility for the inputs.

Dropout may be implemented on any or all hidden layers in the network as well as the visible or input layer. It is not used on the output layer.

In [ ]:
model = tf.keras.models.Sequential([              # model type
  tf.keras.layers.Flatten(input_shape=X_train[1].shape),  # input layer
  tf.keras.layers.Dense(100, activation='relu'),  # hidden layer
  tf.keras.layers.Dropout(0.2),                   # Dropout helps reduce overfitting 
  tf.keras.layers.Dense(10),                      # output to each class, could just stop here
  tf.keras.layers.Softmax()                       # convert to probability
])
#define our optimizer
sgd = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=False, name='SGD')
#
model.compile(optimizer=sgd,
              loss='categorical_crossentropy', #need to define our loss function
              metrics=['accuracy'])

In [ ]:
tstart = tf.timestamp()
history = model.fit(X_train, Y_train, 
                    epochs=epochs,batch_size=batch_size,
                    validation_split = 0.2) # Data for evaluation
total_time = tf.timestamp() - tstart
print("total time %3.3f seconds"%total_time)

In [ ]:
results_test = model.evaluate(X_test, Y_test, batch_size=128,verbose=0)
printAccuracy(history,results_test)

In [ ]:
plot_result(history,results_test)
plt.title("MNIST 1-hidden layer, with dropout, in %3.2f s"%(total_time)) #overwrite the title 
plt.show()

* Without dropout, the test accuracy was 97.8%. 
* With dropout, we are slightly down to 97.6%.  However, the training accuracy is now nearly the same as the test, which is a good sign that we are not overfitting.

## One more thing: initialization
Lets try once more, this time we will **initialize all weights** to **zero** and then train.  
For more on initializers (and there are many options), check out [https://www.tensorflow.org/api_docs/python/tf/keras/initializers](https://www.tensorflow.org/api_docs/python/tf/keras/initializers).

In [ ]:
model = tf.keras.models.Sequential([                      # model type
  tf.keras.layers.Flatten(input_shape=X_train[1].shape),  # input layer
  tf.keras.layers.Dense(100, activation='relu',
                        kernel_initializer='zeros',       # some options: 'random_uniform','zeros', and 'glorot_normal' 
                        bias_initializer='zeros'),        # hidden layer with relu activation, start w/ zeros  
  tf.keras.layers.Dense(10),                              # output to each class, leave this with small initial weights
  tf.keras.layers.Softmax()                               # softmax (probability) activation
])
#define our optimizer
sgd = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=False, name='SGD')
#
model.compile(optimizer=sgd,
              loss='categorical_crossentropy', #need to define our loss function
              metrics=['accuracy'])

In [ ]:
tstart = tf.timestamp()
history = model.fit(X_train, Y_train, 
                    epochs=epochs,batch_size=batch_size,
                    validation_split = 0.2) # Data for evaluation
total_time = tf.timestamp() - tstart
print("total time %3.3f seconds"%total_time)

## Evaluate

In [ ]:
results_test = model.evaluate(X_test, Y_test, batch_size=128,verbose=0)

In [ ]:
printAccuracy(history,results_test)